# Training Korean Text


In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor
# import pycor.korlang as korlang
import pycor.utils as utils
import pycor.scoring as scoring
from pycor.res import WikiResolver
from pycor.res import CollocationResolver
from pycor.res import yongeonresolver as yr

stopwatch = utils.StopWatch()

docsize = 3000

#for aux in lm.auxmap.values():
#    print(aux)

def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = listfiles('../samples')
files = sorted(files)
files.extend( listfiles('../../data/wiki2') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")

outputpath = "../../output/" + str(docsize) + "/"
model_path = outputpath + "model/"

os.makedirs(outputpath, exist_ok=True)
os.makedirs(model_path, exist_ok=True)


dictionary_path = "../samples/dic/dictionary.csv"

# trainer = korlang._trainer

# 사전 등록 
pycor.loaddic(dictionary_path)
# wikiResolver = WikiResolver()
# pycor.addresolver(wikiResolver)

collocationResolver = CollocationResolver()
pycor.addresolver(collocationResolver)

yongeonResolver = yr.YongeonResolver()
pycor.addresolver(yongeonResolver)

Init Trainer
Init DocResolver
3240 files
Loading Dictionary from ../samples/dic/dictionary.csv
Load  ../samples/dic/dictionary.csv   소요시간: 0.005


In [2]:

stopwatch.start()

def extractHeads(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            word_texts.append(word.bestpair.head.text)
        else:
            word_texts.append(word.text)
    return word_texts


def extractTails(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            #word_texts.append(word.bestpair.tail.text + ":" + str(word.bestpair.type))
            word_texts.append(word.bestpair.tail.text)
        else:
            word_texts.append("")
            #word_texts.append(word.text)
    return word_texts



def extractTags(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            word_texts.append(word.bestpair.tags)
        else:
            word_texts.append("*")
    return word_texts



def writeTails(outputpath, index, words_array):
    outputfile = outputpath + 'samples/' + str(index) + '_tails.txt'
    
#     words_array = trainer.scoreDocument(sentence_array)
    
    with open(outputfile, 'w', encoding='utf-8') as out:
        writer = csv.writer(out)
        for words in words_array:
            word_texts = extractTails(words)
            writer.writerow(word_texts)
        out.close()
        
def writeAll(tailswriter, headswriter, tagswriter, words_array):
#     words_array = trainer.scoreDocument(sentence_array)
    
    for words in words_array:
        word_texts = extractTails(words)
        
        if len(word_texts) > 2:
            tailswriter.writerow(word_texts)
            heads = extractHeads(words)
            headswriter.writerow(heads)
            tags = extractTags(words)
            tagswriter.writerow(tags)



        
tailsfile = open(outputpath + 'tails.txt', 'w', encoding='utf-8')
tailswriter = csv.writer(tailsfile)


headsfile = open(outputpath + 'heads.txt', 'w', encoding='utf-8')
headswriter = csv.writer(headsfile)


tagsfile = open(outputpath + 'tags.txt', 'w', encoding='utf-8')
tagswriter = csv.writer(tagsfile)

print("Loading Texts >>> ")
index =0
for file in files[:docsize]:
    if file.endswith(".txt") :
        _, words_array = pycor.readfile(file)
        writeAll(tailswriter,headswriter, tagswriter,words_array)
        index += 1
        if index % 100 == 0:
            print(index,end=">")
            if index % 1000 == 0:
                print()
            
tailsfile.close()
tagsfile.close()

print("Load Texts: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")



Loading Texts >>> 
100>200>300>400>500>600>700>800>900>1000>
1100>1200>1300>1400>1500>1600>1700>1800>1900>2000>
2100>2200>2300>2400>2500>2600>2700>2800>2900>Load Texts:  540s.317ms. ( 540,316 ms.)


In [3]:
def writeWord(writer, word):
    if len(word.particles) == 0:
        writer.writerow([word.text, 'X'])
    for part in word.particles:
        h = part.head.text if part.head else ''
        t = part.tail.text if part.tail else ''
        writer.writerow([word.text, h, t, part.score, part.tags, part.pos])

utils.writecsv(outputpath + "np_words.csv", pycor.getmodel().words.values(), writeWord)

def writeWord(writer, word):
    bestpair = word.bestpair
    if bestpair :
        h = bestpair.head.text if bestpair.head else ''
        t = bestpair.tail.text if bestpair.tail else 'X'
        writer.writerow([ word.text, '   ', h, t, '   ', bestpair.score, bestpair.tags, bestpair.pos ])
    else:
        writer.writerow([ word.text, 'X' ])
        
utils.writecsv(outputpath + "np_words_scored.csv", pycor.getmodel().words.values(), writeWord)

Write  ../../output/3000/np_words.csv   소요시간: 7s.556ms. ( 7,555 ms.)
Write  ../../output/3000/np_words_scored.csv   소요시간: 4s.003ms. ( 4,003 ms.)


In [4]:
import pycor.dev.model as dev

stopwatch.start()

snglist, ylist, clist, ambilist = pycor.buildvocab()

wordmap = pycor.getmodel()

print("Build Vocab: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")
print("Word Count : " , utils.comma(len(wordmap.words)) )
print("head Count : " , utils.comma(len(wordmap.heads)) )
print("tail Count : " , utils.comma(len(wordmap.tails)) )

oldmodel = None

try:
    oldmodel = dev.loadmodel(model_path)
except Exception as e:
    print(e)

if oldmodel:
    print("Old Heads:", len(oldmodel.heads))
    result = dev.compareHeads(oldmodel.heads, wordmap.heads)
    dev.saveCompHeads(model_path+"/compare.csv", result)

pycor.savemodel(model_path)


# with open(model_path + "ambiguous.csv", 'w', encoding='utf-8') as csvfile :
#     writer = csv.writer(csvfile)
#     for head in ambilist:
#         writer.writerow([head.text, '+'.join(head.pos), head.tails])
            
#     csvfile.close()

# with open(model_path + "single.csv", 'w', encoding='utf-8') as csvfile :
#     writer = csv.writer(csvfile)
#     for head in snglist:
#         writer.writerow([head.text, '+'.join(head.pos), head.tails])
            
#     csvfile.close()

Single Count: 185097
용언 Count: 15880
체언 Count: 91513
Ambiguous Count: 8412
Heads Count: 401136
Collocations Count: 61759
Tails Count: 939
Clear words size: 517876
Build Vocab:  8s.263ms. ( 8,263 ms.)
Word Count :  0
head Count :  401,136
tail Count :  939
Load  ../../output/3000/model//heads.csv   소요시간: 5.358
Load  ../../output/3000/model//tails.csv   소요시간: 0.004
Old Heads: 305110
Saving Model to ../../output/3000/model/
Save  ../../output/3000/model//heads.csv   소요시간: 2.497
Save  ../../output/3000/model//tails.csv   소요시간: 0.006
Save  ../../output/3000/model//collocations.csv   소요시간: 0.361
Clear Heads: 115761 remains.


In [5]:

yongeonResolver.writemap(outputpath + "/yongeon.csv")
yongeonResolver.writeindex(outputpath + "/yongeon_index.csv")


Save  ../../output/3000//yongeon.csv   소요시간: 0.841
Save  ../../output/3000//yongeon_index.csv   소요시간: 0.152


In [6]:
def writeTails(writer, tail):
    #row = [tail.text]
    #for head in tail.heads:
    #    row.append(head.text)
    #row = [tail.text, len(tail.heads), tail.heads]
    row = [tail.text, len(tail.heads)]
    writer.writerow(row)

wordmap = pycor.getmodel()
tailsorted = sorted(wordmap.tails.values(), key=lambda tail:tail.text[len(tail.text)-1])
utils.writecsv(outputpath + "np_tails.csv", tailsorted, writeTails)


tailsorted2 = sorted(wordmap.tails.values(), key=lambda tail:len(tail.heads), reverse=True)
utils.writecsv(outputpath + "np_tails_occ.csv", tailsorted2, writeTails)



Write  ../../output/3000/np_tails.csv   소요시간: 0s.014ms. ( 14 ms.)
Write  ../../output/3000/np_tails_occ.csv   소요시간: 0s.007ms. ( 7 ms.)


In [7]:
def writeHeads(writer, head):
    row = [ str(head), '  ',head.score, ' ', len(head.tails), ' ']
    for tail in head.tails:
        row.append(tail.text)
    writer.writerow(row)

wordmap = pycor.getmodel()

headsorted = sorted(wordmap.heads.values(), key=lambda head:head.text)
utils.writecsv(outputpath + "np_heads.csv", headsorted, writeHeads)


headsorted2 = sorted( wordmap.heads.values(), key=lambda head:len(head.tails), reverse=True)
utils.writecsv(outputpath + "np_heads_occ.csv", headsorted2, writeHeads)


headset = set()

for word in  wordmap.words.values():
    bestpair = word.bestpair
    if bestpair and bestpair.head:
        headset.add(bestpair.head)
        
def writeHead(writer, head):
    row = [ str(head), '\t', len(head.tails), '\t']
    
    for tail in head.tails:
        row.append(tail.text)
        
    writer.writerow(row)
        

        
headlist = sorted(headset , key=lambda head:head.text )

utils.writecsv(outputpath + "np_head_scored.csv", headlist, writeHead)




Write  ../../output/3000/np_heads.csv   소요시간: 0s.590ms. ( 590 ms.)
Write  ../../output/3000/np_heads_occ.csv   소요시간: 0s.517ms. ( 516 ms.)
Write  ../../output/3000/np_head_scored.csv   소요시간: 0s.016ms. ( 15 ms.)


In [8]:

def extractWords(words) :
    word_texts = []
    for word in words:
        if word.bestpair:
            #word_texts.append(word.bestpair.head.text +":"+word.bestpair.tail.text)
            pos = word.bestpair.pos if word.bestpair.pos else ''
            tag = word.bestpair.tags if word.bestpair.tags else ''
#             print(word.bestpair.tail.occurence())
            postag = None
            if pos or tag:
                postag = "("+ str(pos) + ":" + str(tag) +")"
            else :
                postag = ""
            word_texts.append(word.bestpair.head.text + postag)
        else:
            word_texts.append(word.text)
    return word_texts

def writeExtracted(outputpath, suffix, index, words_array):
    outputfile = outputpath + str(index) + suffix + '.txt'
    
#     words_array = trainer.scoreDocument(sentence_array)

    with open(outputfile, 'w', encoding='utf-8') as out:
        writer = csv.writer(out)
        for words in words_array:
            word_texts = extractWords(words)
            writer.writerow(word_texts)
        out.close()
        
index = 0
for file in files[:5]:
    if file.endswith(".txt") :
        sentence_array, words_array = pycor.readfile(file)
        index += 1
        writeExtracted(outputpath, '_train', index, words_array)

import pycor
pycor.loadmodel(model_path)

index = 0
for file in files[:5]:
    if file.endswith(".txt") :
        sentence_array, words_array = pycor.readfile(file)
        index += 1
        writeExtracted(outputpath, '_pycor', index, words_array)




Loading Model from ../../output/3000/model/
Load  ../../output/3000/model//heads.csv   소요시간: 5.119
Load  ../../output/3000/model//tails.csv   소요시간: 0.003


In [9]:
word = pycor.resolveword('라부아지에', debug=True)
print(word.bestpair)
word = pycor.resolveword('라부아지에에게')
print(word.bestpair)
word = pycor.resolveword('라부아지에가', debug=True)
print(word.bestpair)
word = pycor.resolveword('라부아지에를')
print(word.bestpair)
word = pycor.resolveword('라부아지에', debug=True)
print(word.bestpair)

word = pycor.resolveword('간디에')
print(word.bestpair)
word = pycor.resolveword('간디에게')
print(word.bestpair)

라부아지에 라부아지에  0.999052577884027 [] set()
라부아지에 라부아지 에 0.9999210557828495 ['JKB-TO'] set()
라부아지에 라부아지 에 0.9999210557828495 ['JKB-AT'] set()
라부아지:에:{'JKB-TO', 'V', 'MM'}:17792
라부아지에:에게:{'JKB-TO', 'MM'}:2039
라부아지에가 라부아지에 가 0.9998714373905285 ['JKS'] set()
라부아지에가 라부아지 에가 0.9999893103993414 ['JKS', 'JKB-TO'] set()
라부아지에가 라부아지에 가 0.9998714373905285 ['JKC'] set()
라부아지:에가:{'JKB-TO', 'JKS'}:15
라부아지에:를:{'ETM', 'JKO'}:11371
라부아지에 라부아지에  0.9998716444651541 [] set()
라부아지에 라부아지 에 0.9999893158493547 ['JKB-TO'] set()
라부아지에 라부아지 에 0.9999893158493547 ['JKB-AT'] set()
라부아지:에:{'JKB-TO', 'V', 'MM'}:17792
간디:에:{'JKB-TO', 'V', 'MM'}:17793
간디:에게:{'JKB-TO', 'MM'}:2040


In [10]:
word = pycor.resolveword('감소되서')
print(word.bestpair)
word = pycor.resolveword('감소돼서')
print(word.bestpair)
word = pycor.resolveword('감소되었다가')
print(word.bestpair)
word = pycor.resolveword('감소됐다가', debug=True)
print(word.bestpair)

감소되:서:{'EC-because'}:10566
감소되:어서:{'ADJ|V+IRB', 'V', 'EC-because', 'MM', 'V+IRD+AMBI'}:244
감소되:었다가:{'V', 'V+IRD+AMBI', 'EPT-pp', 'EC-and'}:97
감소됐다가 감소됐 다가 0.9241418199787566 ['EC-and'] set()
감소됐다가 감소되 었다가 1.0 ['EC-and', 'EPT-pp'] set()
감소됐다가 감소됐다 가 0.8175744761936437 ['JKS'] set()
감소됐다가 감소됐다 가 0.8175744761936437 ['JKC'] set()
감소되:었다가:{'V', 'V+IRD+AMBI', 'EPT-pp', 'EC-and'}:97
